# Movie Recommendation System Using OpenAGI

This notebook leverages the OpenAGI framework to create an interactive movie recommendation system.


## Installation

Before running the script, ensure you have the OpenAGI library installed. You can install it using the following command:

In [ ]:
pip install openagi

In [ ]:

from openagi.actions.files import WriteFileAction, ReadFileAction
from openagi.actions.tools.ddg_search import DuckDuckGoSearch
from openagi.actions.tools.webloader import WebBaseContextTool
from openagi.agent import Admin
from openagi.llms.azure import AzureChatOpenAIModel
from openagi.memory import Memory
from openagi.planner.task_decomposer import TaskPlanner
from openagi.worker import Worker
from rich.console import Console
from rich.markdown import Markdown


## Environment Setup
The script begins by setting up the necessary environment variables for Azure OpenAI services. This includes specifying the base URL, deployment name, model name, API key, and API version. These configurations are essential for authenticating and accessing the Azure OpenAI services.

In [ ]:
if __name__ == "__main__":
    import os

    os.environ["AZURE_BASE_URL"] = " "
    os.environ["AZURE_DEPLOYMENT_NAME"] = " "
    os.environ["AZURE_MODEL_NAME"] = " "
    os.environ["AZURE_OPENAI_API_KEY"] = " "
    os.environ["AZURE_OPENAI_API_VERSION"] = " "

    config = AzureChatOpenAIModel.load_from_env_config()
    llm = AzureChatOpenAIModel(config=config)

## Workers used:

1. User Input Collector: The User Input Collector is responsible for gathering movie preferences from the user. It prompts the user to provide 2-3 movies they enjoy and asks for specific genres or themes associated with those films. This component ensures that the input is confirmed before proceeding to the recommendation phase.

2. Movie Recommender: The Movie Recommender utilizes the user’s preferences to suggest similar movies. It analyzes the input from the User Input Collector, searches for related films using the DuckDuckGo search tool, and ranks the recommendations based on similarity and popularity. The output includes a structured list of recommended movies with descriptions.

3. Recommendation Review Specialist: The Recommendation Review Specialist reviews the list of recommended movies to ensure clarity and relevance. This component checks the descriptions for engagement, suggests additional movies, and finalizes the presentation format for user readability.

In [ ]:
    user_input_collector = Worker(
        role="User Input Collector",
        instructions="""
        Your task is to collect movie preferences from the user. Follow these steps:

        1. Prompt the user to provide 2-3 movies they enjoy.

        2. Ensure the user specifies the genres or themes they like about these movies.

        3. Collect the input and prepare it for the recommendation process.

        4. Confirm the collected preferences with the user before proceeding.

        Your output should be the list of movies and their associated genres or themes.
        """,
        actions=[
            DuckDuckGoSearch,
            WebBaseContextTool,
        ]
    )

    recommender = Worker(
        role="Movie Recommender",
        instructions="""
        As a Movie Recommender, your task is to suggest movies based on the user's preferences. Follow these steps:

        1. Receive the list of movies and genres/themes from the User Input Collector.

        2. Use DuckDuckGoNewsSearch to find similar movies based on the provided input.

        3. Analyze the search results for related films that match the user's tastes.

        4. Rank the recommendations based on similarity and popularity.

        5. Prepare a summary of the recommended movies, including a brief description of each.

        6. Present the recommendations to the user in an engaging and informative manner.

        Your output should be a structured list of recommended movies with descriptions.
        """,
        actions=[
            DuckDuckGoSearch,
            WebBaseContextTool,
        ]
    )

    reviewer = Worker(
        role="Recommendation Review Specialist",
        instructions="""
        As a meticulous reviewer, your task is to ensure the recommendations are relevant and well-presented. Follow these steps:

        1. Review the list of recommended movies provided by the Movie Recommender.

        2. Check for clarity: Ensure that each movie description is clear and engaging.

        3. Suggest improvements: Recommend any additional movies that may fit the user's preferences.

        4. Finalize the presentation: Ensure the output is formatted correctly for user readability.

        Your output should be the final list of recommended movies with any suggested enhancements.
        """,
        actions=[
            DuckDuckGoSearch,
            WebBaseContextTool,
        ]
    )


## Admin
The Admin  orchestrates the workflow by assigning tasks to the various workers. It manages the interaction between the user input collector, recommender, and reviewer, ensuring a smooth flow of information and user engagement.

In [ ]:
admin = Admin(
        planner=TaskPlanner(human_intervene=True),
        memory=Memory(),
        llm=llm,
    )

admin.assign_workers([user_input_collector, recommender, reviewer])


## Execution
Finally, the script runs the agent to collect user preferences and provide movie recommendations. The results are printed in a structured format, enhancing readability and user engagement.

In [ ]:
   res = admin.run(
        query="Recommend movies based on user preferences.",
        description="""
        The user will provide 2-3 movies they like along with their genres or themes.
        Your task is to recommend similar movies based on this input.
        Ensure to collect the user's preferences first and then provide a list of recommendations.
        """
    )

Agent: What specific genres or themes should I focus on when recommending movies based on the user's input?
You: thriller and crime
Agent: What specific movies should I ask the user for to gather their preferences?
You: movies with suspense and thriller
Agent: Could you specify what types of suspense and thriller elements the user prefers, such as psychological, action-packed, or character-driven?
You: psychological 
Agent: What specific 2-3 movies should I ask the user for to gather their preferences in the thriller and crime genres?
You: inception,parasite,shutter island
Agent: What specific criteria should I use to determine if the recommended movies are similar to the user's preferences based on the provided movies?
You: english movies
Agent: Are there any specific elements or themes within thriller and crime movies, aside from psychological aspects, that the user prefers, such as character development or plot twists?
You: no


Tasks: [{'task_name': 'Collect User Movie Preferences'

## Conclusion
This demonstrates how to utilize the OpenAGI framework for building an interactive movie recommendation tool. By collecting user preferences and leveraging search capabilities, it provides personalized movie suggestions that enhance the user experience.

In [ ]:
print("-" * 100)
Console().print(Markdown(res))

----------------------------------------------------------------------------------------------------


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                               Recommended Movies                                                ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Here are some movies you might enjoy:                                                                              

 1 Inception                                                                                                       
   A mind-bending thriller that explores the world of dreams and the subconscious.                                 
 2 The Shawshank Redemption                                                                                        
   A powerful story of hope and friendship set against the backdrop of a maximum-security prison.                  
 3 The Godfather                                                                                                   
   An iconic tale of family, power, and betrayal within the Italian-American mafia.                                
 4 Parasite                                                                                                        
   A darkly comedic thriller that examines class disparity through the lives of two families.                      
 5 Interstellar                                                                                                    
   A visually stunning sci-fi epic about love, sacrifice, and the survival of humanity.                            
 6 The Dark Knight                                                                                                 
   A gripping superhero film that delves into morality through the conflict between Batman and the Joker.